In [ ]:
''' 
Authors: Balachander S, Yogesh Chandra Singh Samant, Prahalad Srinivas C G, B Varshin Hariharan
'''
import numpy as np
import pandas as pd
import time
from collections import Counter

def histogram_sampler(data, no_new_data, data_feat, preserve):

    if (data_feat == 'c'):
        start_time = time.time()
        print('Existing data:', len(data))
        print('New data to be produced:', no_new_data)

        '''Function parameters'''

        '''
        X_new = New data for each iteration
        len_X_new = length of the newly generated data
        iter_count = no. of iterations
        data_gen = Augmented data (original data + newly generated data)
        '''
        X_new = []
        len_X_new = len(X_new)
        iter_count = 0
        data_gen = data

        while (len_X_new < 0.7*no_new_data):
            iter_count+=1
            print('-----\niter_count=',iter_count)

            '''
            Histogram: Generating the histogram , choosing the mid-value of the bins, and normalizing frequency
            'fd'- Freedman–Diaconis rule is employed to choose the bin size, as it depends on the spread of the data, without any presumption
            '''

            if(iter_count == 1):
                Y,X_interval=np.histogram(data_gen,bins='doane')
                n_bins = len(Y)

            else:
                Y,X_interval=np.histogram(data_gen,bins=n_bins)

            X = ((X_interval[0:-1] + X_interval[1:])/2) 
            Y = Y/max(Y)

            bin_val = list(np.round(X,8))
            weight = list(Y)
            hist = dict(zip(bin_val,weight))

            for xi in bin_val[0:-1]:

                '''
                Values: choosing the values for undergoing validity check
                '''

                bin_width = ((max(bin_val) - min(bin_val)) / int(len(bin_val)-1))
                xm = xi + (bin_width/2)
                x1 = xi
                y1 = hist[xi]

                res = None
                temp = iter(hist)
                for key in temp:
                    if(key == xi):
                        res = next(temp,None)

                y2 = hist[res]
                ym = ((y1+y2)/2)

                '''
                Validity check: checking if the specified value can be considered
                '''

                '''if(no_new_data <= len(data)):
                    ym = ym*(np.random.rand()<=ym)
                    y1 = y1*(np.random.rand()<=y1)
                '''
                #else:
                ym = ym*(abs(np.random.normal(0,0.5))<=ym)
                y1 = y1*(abs(np.random.normal(0,0.5))<=y1)

                '''
                Appending: appending the valid values
                '''

                if (ym!=0):
                    X_new.append(np.round(xm,8))
                    #X_new.append(np.round(xm+0.1*xm,8))
                    #X_new.append(np.round(xm-0.1*xm,8))
                if (y1!=0):
                    X_new.append(np.round(x1,8))
                    #X_new.append(np.round(x1+0.1*xm,8))
                    #X_new.append(np.round(x1-0.1*xm,8))

            '''
            Stopping: bins * 2, length check
            '''

            data_gen = data_gen + X_new
            n_bins = n_bins*2     
            len_X_new+= len(X_new)
            print(len_X_new)
            X_new = []
            print("--- %s seconds ---" % (time.time() - start_time))


        print(len(data_gen)-len(data),no_new_data)

        if(len(data_gen)-len(data) >= no_new_data):
            data_gen = data_gen[:len(data)] + list(np.random.choice(data_gen[len(data):], no_new_data, replace = False))
            print('\nNew data generated:', len(data_gen[len(data):]), '\nNew data:', len(data_gen), '\n')
            #sns.distplot(data_gen)
            if(preserve == False):
                data_gen = data_gen[len(data):]
            return data_gen

        else:
            print('to discrete...', no_new_data - (len(data_gen)-len(data)))
            samples = histogram_sampler(data_gen, no_new_data - (len(data_gen)-len(data)), 'd', preserve = True)
            if(preserve == False):
                samples = samples[len(data):]
            return samples
        
    
    
    elif(data_feat == 'd'):
        X_new=[]
        data_gen=[]
        disc_data= list(set(data))

        for i in disc_data: 
            x=data.count(i)
            X_new.append(round(x*(no_new_data) / len(data)))
        #print(x_new,sum(x_new))

        for j in range(0,len(X_new)):
            for i in range(X_new[j]):
                data_gen.append(disc_data[j])

        if(len(data_gen)==0):
            data_gen = data + data_gen

        print(no_new_data, sum(X_new))            
        if(no_new_data > sum(X_new)):
            data_gen = data + data_gen
            data_gen = list(data_gen + list(np.random.choice(data_gen,int(no_new_data-sum(X_new)),replace = False)))    

        data_gen = list(np.random.choice(data_gen,int(no_new_data),replace = False))   
        
        if(preserve == True):
            data_gen = data + data_gen
    
        #sns.distplot(data_gen)     
        print('\nNew data generated:', len(data_gen[len(data):]), '\nNew data:', len(data_gen), '\n') 
        return data_gen
    
    else:
        print('NA')

def label_split(df_class, diff, label_name, cd, label_column, preserve):
    cdi = 0
    df_ = pd.DataFrame(columns=[])
    del df_class[label_column]
    for (columnName, columnData) in df_class.iteritems(): 
        print(columnName)
        feat_type = cd[cdi]
        df_[columnName] = histogram_sampler(list(columnData.values), diff, feat_type, preserve)
        cdi+=1
    df_[label_column] = label_name
    print(df_)

    return df_

def class_balance(data, label_column, cd, augment = False, preserve = True):
    split_list=[]
    #label_column = 'species'
    for label, df_label in data.groupby(label_column):
        split_list.append(df_label)

    maxLength = max(len(x) for x in split_list)

    if(augment == False):
        augmented_list=[]
        for i in range(0,len(split_list)):

            label_name = list(set(split_list[i][label_column]))[0]
            diff = maxLength - len(split_list[i])
            augmented_list.append(label_split(split_list[i], diff, label_name, cd, label_column, preserve))
        finaldf = pd.DataFrame(columns=[])

    elif(type(augment) == dict):
        augmented_list=[]
        for i in range(0,len(split_list)):

            label_name = list(set(split_list[i][label_column]))[0]
            diff = augment[label_name]
            augmented_list.append(label_split(split_list[i], diff, label_name, cd, label_column, preserve))
        finaldf = pd.DataFrame(columns=[])

    elif(type(augment) == int):

        label_count = dict(Counter(list(df[label_column])))
        count_key = list(label_count.keys())
        count_val = list(label_count.values())

        for i in range(0,len(count_val)):
            count_val[i] = round(count_val[i] * augment  /sum(list(label_count.values())))


        while(sum(count_val) != augment):
            if(sum(count_val) > augment):
                rand_indx = int(np.random.rand() * len(count_val))
                if(count_val[rand_indx] > 0):
                    count_val[rand_indx]-= 1

            else:
                rand_indx = int(np.random.rand() * len(count_val))
                count_val[rand_indx]+= 1

        new_count = dict(zip(count_key, count_val))

        augmented_list=[]
        for i in range(0,len(split_list)):

            label_name = list(set(split_list[i][label_column]))[0]
            diff = new_count[label_name]
            augmented_list.append(label_split(split_list[i], diff, label_name, cd, label_column, preserve))
        finaldf = pd.DataFrame(columns=[])


        #finaldf = class_balance(data, label_column, cd, augment = new_count, preserve = preserve)


    for i in range(0,len(split_list)):
        finaldf = pd.concat([finaldf,augmented_list[i]],axis=0)

    return finaldf

In [ ]:
df = pd.read_csv('glass.csv')
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [ ]:
df.drop('ID',axis=1,inplace=True)
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [ ]:
print(df['Type'].value_counts())

2    76
1    70
7    29
3    17
5    13
6     9
Name: Type, dtype: int64


In [ ]:
start_time = time.time()
cd = []

for i in range(0,7):
    cd.append('c')
for i in range(0,2):
    cd.append('d')

print(cd)
a_df = class_balance(df, label_column = 'Type', cd = cd, augment = False, preserve = True)
print("\n\n\n>>>>>>>>> %s seconds " % (time.time() - start_time))
a_df

['c', 'c', 'c', 'c', 'c', 'c', 'c', 'd', 'd']
RI
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
7
--- 0.002422332763671875 seconds ---
7 6

New data generated: 6 
New data: 76 

Na
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
7
--- 0.002316713333129883 seconds ---
7 6

New data generated: 6 
New data: 76 

Mg
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
5
--- 0.0016393661499023438 seconds ---
5 6
to discrete... 1
1 0

New data generated: 1 
New data: 76 

Al
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
8
--- 0.0022351741790771484 seconds ---
8 6

New data generated: 6 
New data: 76 

Si
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
8
--- 0.0023069381713867188 seconds ---
8 6

New data generated: 6 
New data: 76 

K
Existing data: 70
New data to be produced: 6
-----
iter_count= 1
5
--- 0.0010159015655517578 seconds ---
5 6
to discrete... 1
1 0

New data generated: 1 
New data: 76 

Ca
Existi

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.521010,13.640000,4.490000,1.100000,71.780000,0.060000,8.750000,0.00,0.00,1
1,1.517610,13.890000,3.600000,1.360000,72.730000,0.480000,7.830000,0.00,0.00,1
2,1.516180,13.530000,3.550000,1.540000,72.990000,0.390000,7.780000,0.00,0.00,1
3,1.517660,13.210000,3.690000,1.290000,72.610000,0.570000,8.220000,0.00,0.00,1
4,1.517420,13.270000,3.620000,1.240000,73.080000,0.550000,8.070000,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...
71,1.515080,13.336667,0.000000,2.276429,73.219688,0.000000,9.070000,0.64,0.00,7
72,1.517191,14.830000,0.000000,2.457500,73.335000,0.072656,8.530000,1.55,0.00,7
73,1.516230,14.200000,0.417500,1.220179,72.720000,0.525000,8.978194,1.71,0.09,7
74,1.516020,14.616667,0.023194,2.220000,73.335000,0.075000,8.557222,0.56,0.00,7


In [ ]:
a_df.to_csv('HAT_Glass.csv')

In [ ]:
print(a_df['Type'].value_counts())

7    76
6    76
5    76
3    76
2    76
1    76
Name: Type, dtype: int64
